In [ ]:
%%capture google_tran_installation
!pip install googletrans
!pip install --upgrade googletrans==4.0.0-rc1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import pprint
project_dir = '/content/drive/MyDrive/Bert-Files/Topic Modelling'
os.chdir(project_dir)

In [ ]:
from googletrans import Translator
def translate_text(text, source_lang='ms', target_lang='en'):
    translator = Translator(service_urls=['translate.google.com'])
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text

In [ ]:
with open('data.txt','r') as file:
    data = file.read()

# direct malay text preprocessing
docs = data.split('\n')
final_docs =[]
for i in docs:
    i = i.replace("'",'')
    i = i.replace(",",'')
    i = i.replace('"','')
    i = i.replace('*','')
    i = i.replace('(','')
    i = i.replace(')','')
    i = i.replace('/','')
    i = i.replace('\\','')
    i = i.replace(':','')
    i = i.replace('.','')
    i = i.replace('-','')
    i = i.replace('&','')
    i = re.sub(r'[^a-zA-Z\s]', '', i)
    i = i.strip()
    if len(i)>0:
        final_docs.append(i)


# converting malay to english and preprocessing
docs_english = []
for i in data.split('\n'):
    # print('Malay = ',i)
    try:
        translated = translate_text(i)
        # print('English = ', translated)
        docs_english.append(translated)
    except Exception as e:
        # print('Erorr Occured !',e)
        pass
    # print('------------------------------------------')

final_docs_english =[]
for i in docs_english:
    i = i.replace("'",'')
    i = i.replace(",",'')
    i = i.replace('"','')
    i = i.replace('*','')
    i = i.replace('(','')
    i = i.replace(')','')
    i = i.replace('/','')
    i = i.replace('\\','')
    i = i.replace(':','')
    i = i.replace('.','')
    i = i.replace('-','')
    i = i.replace('&','')
    i = re.sub(r'[^a-zA-Z\s]', '', i)
    i = i.strip()
    if len(i)>0:
        final_docs_english.append(i)

In [ ]:
final_docs = [i.lower() for i in final_docs]

In [ ]:
final_docs_english = [i.lower() for i in final_docs_english]

In [ ]:
%%capture
!pip install bertopic

In [ ]:
from bertopic import BERTopic
from bertopic.backend import languages

In [ ]:
'malay' in languages

True

In [ ]:
%%capture model_loading
# fit transforming model with original malay text
model = BERTopic(language="malay")
topics, probs = model.fit_transform(final_docs)

In [ ]:
# model.get_topic_info()

In [ ]:
for i in [-1,0,1]:
    pprint.pprint(model.get_topic(i))

False
[('bb', 0.10747747930599626),
 ('mei', 0.07400650175794449),
 ('bayaran', 0.06624505296401538),
 ('umt', 0.05798849286695409),
 ('di', 0.049130587195329936),
 ('prog', 0.049130587195329936),
 ('program', 0.049130587195329936),
 ('kejohanan', 0.04442913481136869),
 ('hadiah', 0.04442913481136869),
 ('pelajar', 0.04442913481136869)]
[('blk', 0.3501274002339871),
 ('bkln', 0.3185279839176395),
 ('dn', 0.3185279839176395),
 ('mesy', 0.2899318837596284),
 ('penginapanmkn', 0.21146070299611602),
 ('stpm', 0.21146070299611602),
 ('mkn', 0.2052290189173162),
 ('bg', 0.1996684862499987),
 ('pnylrsn', 0.17005510281309658),
 ('kerja', 0.16390748383113043)]


In [ ]:
# model.get_document_info(final_docs)

In [ ]:
%%capture model_loading
# fit_transform model with converted(english) docs
model_english = BERTopic(language="english")
topics, probs = model_english.fit_transform(final_docs_english)

In [ ]:
model_english.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,36,-1_and_bb_port_umt,"[and, bb, port, umt, to, benchmark, purchase, ...","[Training and Meetings in Greece Me, through ..."
1,0,44,0_bb_community_may_program,"[bb, community, may, program, championship, pr...","[Johor High Community Community Program, BB Pr..."
2,1,20,1_bkln_dn_blk_mesy,"[bkln, dn, blk, mesy, bg, mkn, pnylrsn, stpm, ...",[bkln mkn dn mnm bg mesy pnylrsn special work ...
3,2,18,2_payment_of_service_inv,"[payment, of, service, inv, services, machine,...","[Payment of SN, Payment of SN Photostat Machin..."


In [ ]:
model_english.get_topic(0)

[('bb', 0.21140888295392452),
 ('umt', 0.12922129844122435),
 ('prize', 0.09663841833215636),
 ('championship', 0.09663841833215636),
 ('esailing', 0.0664180940363867),
 ('sports', 0.0664180940363867),
 ('may', 0.058537111083521505),
 ('student', 0.057983050999293816),
 ('the', 0.052500253692609036),
 ('of', 0.05020601198780516)]

### Part 2

In [ ]:
import collections
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# removing extra white spaces inside a doc
def unify_whitespaces(text):
    cleaned_string = re.sub(' +', ' ', text )
    return cleaned_string

In [ ]:
!pip install nltk

In [ ]:
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#  stopwords removal
from nltk.stem import PorterStemmer

stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop ]
    return ' '.join(text)

In [ ]:
# stemming
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def Stemming(text):
    stem = []
    stopword = stopwords.words('english')
    snowball_stemmer = SnowballStemmer('english')
    word_tokens = nltk.word_tokenize(text)
    stemmed_word = [ snowball_stemmer.stem(word) for word in word_tokens ]
    stem = ' '.join(stemmed_word)
    return stem

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
%%capture
def embed(model, model_type, sentences ):
    if model_type == 'use':
        embeddings  = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences)

    return embeddings

model_st1 = SentenceTransformer('all-mpnet-base-v2', device='cpu' )

In [ ]:
embeddings_st1 = embed(model_st1, 'sentence transformer', final_docs_english )

In [ ]:
import umap # dimensionality reduction
import hdbscan # clustering
from functools import partial

# To perform the Bayesian Optimization for searching the optimum hyperparameters,
# we use hyperopt package:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

In [ ]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,32)),
    "n_components": hp.choice('n_components', range(3,32)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,5)),
    "random_state": 42
}

label_lower = 10
label_upper = 100
max_evals = 25

In [ ]:
import utils

In [ ]:
best_params_use, best_clusters_use, trials_use = utils.bayesian_search(
    embeddings_st1,
    space=hspace,
    label_lower=label_lower,
    label_upper=label_upper,
    max_evals=max_evals
)

100%|██████████| 25/25 [00:24<00:00,  1.04trial/s, best loss: 0.059322033898305086]
best:
{'min_cluster_size': 2, 'n_components': 30, 'n_neighbors': 7, 'random_state': 42}
label count: 15


In [ ]:
import pandas as pd
data_clustered = pd.DataFrame(
    data = list(zip(final_docs_english, best_clusters_use.labels_ )) ,
    columns=['text', 'label_st1' ]
)

In [ ]:
from collections import Counter

sent_with_word_lemma = []

for intent in final_docs_english:
    doc = nlp(intent)
    temp_sentence = ""
    this_one = False
    for token in doc:
        if (token.pos_ in ['VERB', 'NOUN', 'ADJ'] or (token.dep_ == 'dobj' ) ):
            temp_sentence += token.lemma_.lower() + " "
    sent_with_word_lemma.append(temp_sentence)

In [ ]:
import numpy as np
def compute_IDF(documents):
    word_count = Counter()
    for doc in documents:
        if 'drops(players' in doc:
            print(doc)
        word_set = set(doc.split())
        word_count.update(word_set)
    # print(word_count)
    total = sum(word_count.values())
    word_IDF = {k : round(( np.log2(total / v ) )) for k, v in word_count.items() }
    return word_IDF

word_IDF = compute_IDF(sent_with_word_lemma)

In [ ]:
def get_group(df, category_col, category ):
    single_category = df[df[category_col] == category ].reset_index(drop = True)
    return single_category

def most_common(lst, n_words ):
    counter = collections.Counter(lst)
    for k in list(counter):
        if counter[k] == 1:
            pass
        else:
            counter[k] *= word_IDF[k]
    return counter.most_common(n_words)

In [ ]:
def extract_labels(category_doc, print_word_counts = False):
    verbs = []
    dobjs = []
    nouns = []
    adjs = []
    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    for i in range(len(category_doc)):
        doc = nlp(category_doc[i])
        for token in doc:
            if (token.is_stop == False ) and (len(str(token).strip()) >0 ):
                if token.pos_ == 'VERB':
                    verbs.extend([token.lemma_.lower()])

                elif token.dep_ == 'dobj':
                    dobjs.extend([token.lemma_.lower()])

                elif token.pos_ == 'NOUN':
                    nouns.extend([token.lemma_.lower()])

                elif token.pos_ == 'ADJ':
                    adjs.extend([token.lemma_.lower()])

    if print_word_counts:
        for word_lst in [verbs, dobjs, nouns, adjs]:
            counter = collections.Counter(word_lst)
            print(counter)

    if len(verbs) > 0:
        verb = most_common(verbs, 1)[0][0]

    if len(dobjs) > 0:
        dobj = most_common(dobjs, 1)[0][0]

    if len(nouns) > 0:
        noun1 = most_common(nouns, 1)[0][0]

    if len(set(nouns)) > 1:
        noun2 = most_common(nouns, 2)[1][0]


    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]

    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)

    if '' in label_words:
        label_words.remove('')

    label = '_'.join(label_words)

    return label

In [ ]:
def apply_and_summarize_labels(df, category_col ):
    numerical_labels = df[category_col].unique()
    label_dict = {}
    for label in numerical_labels:
        current_category = list(get_group(data_clustered, category_col, label )['text'])
        label_dict[label] = extract_labels(current_category)
    summary_df = (
        df.groupby(category_col)['text'].count()
            .reset_index()
            .rename(columns={'text' : 'count' })
            .sort_values('count', ascending = False )
        )

    summary_df['label'] = summary_df.apply(lambda x: label_dict[x[category_col]] , axis = 1 )
    return summary_df

In [ ]:
cluster_summary = apply_and_summarize_labels(data_clustered, 'label_st1' )

In [ ]:
cluster_summary

,label_st1,count,label
9,8,21,esaile_tournament_championship_fee
3,2,20,apr_payment_service
1,0,12,umtprsb_kginner_bkln_s
5,4,10,repair_clubhouse_accommodation_bkln
2,1,8,enter_mei_let_field
13,12,8,itorienteere_kuantan_promotion_polycc
0,-1,7,visit_port_student
14,13,7,sample_studentspd_community_program
7,6,5,sponsor_benchmark_phase
12,11,5,need_college_dining_food
